In [1]:
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
import math
import random
from sklearn import preprocessing
from statsmodels.tsa.arima.model import ARIMA

In [2]:
data = pd.read_excel('zone_merged_reordered_136.xlsx')
# data = pd.read_csv('Zone1_Sur_combine.csv')
# 使用每列的众数填充该列的缺失值
for column in data.columns:
    data[column].fillna(data[column].mode()[0], inplace=True)


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
global data_np
data_np = data.to_numpy()[:, :]
min_max_scaler = preprocessing.MinMaxScaler()
data_np_scale = min_max_scaler.fit_transform(data_np)
#取最后一列
original_chlo_data = data_np[:, -1]
global max_chlo
max_chlo = np.max(original_chlo_data)
chlo_data = data_np_scale[:, -1]

In [4]:
"Optimal parameters"
global feat_num, use_len
feat_num = data_np.shape[1]
use_len = 12
pred_len = 12
hidden_dim = 128
batch_size = 128


def create_index_set(rain_data, data_np, use_len, pred_len):
    sample_size = len(rain_data) - (use_len + pred_len - 1)

    X_sample = np.zeros((sample_size, use_len, data_np.shape[1]))
    Y_sample = np.zeros((sample_size, pred_len))

    for i in range(use_len, len(rain_data) - pred_len + 1):
        Y_sample[i - use_len] = rain_data[i:i + pred_len]
        X_sample[i - use_len] = data_np_scale[i - use_len:i, :]

    X_sample = X_sample.reshape(len(X_sample), use_len, data_np.shape[1])
    Y_sample = Y_sample.reshape(len(Y_sample), pred_len, 1)

    return X_sample, Y_sample

X_sample,Y_sample = create_index_set(original_chlo_data,data_np_scale,use_len,pred_len)
X_tr_sample,X_val,Y_tr_sample,Y_val = train_test_split(X_sample,Y_sample,test_size=0.1,random_state=42)
X_tr,X_1fold,Y_tr,Y_1fold = train_test_split(X_tr_sample,Y_tr_sample,test_size=0.33,random_state=42)
X_2fold,X_3fold,Y_2fold,Y_3fold = train_test_split(X_tr,Y_tr,test_size=0.5,random_state=42)

X1fold_tensor=torch.from_numpy(X_1fold).to(device)
Y1fold_tensor=torch.from_numpy(Y_1fold).to(device)

X2fold_tensor=torch.from_numpy(X_2fold).to(device)
Y2fold_tensor=torch.from_numpy(Y_2fold).to(device)

X3fold_tensor=torch.from_numpy(X_3fold).to(device)
Y3fold_tensor=torch.from_numpy(Y_3fold).to(device)

In [5]:
class RainTrainDataset(Dataset):

    def __init__(self, X_train, Y_train):
        self.input = torch.from_numpy(X_train).to(device)
        self.output = torch.from_numpy(Y_train).to(device)

    def __getitem__(self, index):
        return self.input[index], self.output[index]

    def __len__(self):
        return len(self.input)


class ChloBLSTMGRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=5):
        super(ChloBLSTMGRU, self).__init__()
        self.blstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.blstm(x)
        out, _ = self.gru(out)
        out = self.fc(out[:, -1, :])  # Take the output from the last time step
        out=out*max_chlo
        return out

def evaluate_model(set_tensor, tar_tensor, model, criterion):
    model.eval()
    output = model(set_tensor.float())
    # Reshape, automatically calculate dimensions, convert to (m*n, 1)
    res_output = output.reshape(-1, 1).to(device)
    target = tar_tensor.reshape(-1, 1).to(device)
    MSE = criterion(res_output, target)
    MAE = mean_absolute_error(target.cpu().numpy(), res_output.cpu().numpy())
    R2 = r2_score(target.cpu().numpy(), res_output.cpu().numpy())
    return tar_tensor.cpu().numpy(), output.cpu().numpy(), MSE.item(),MAE, R2

In [6]:
# Instantiate the ChloBLSTMGRU model
model = ChloBLSTMGRU(input_size=feat_num, hidden_size=hidden_dim, output_size=pred_len).to(device)

# Define loss function and optimizer
criterion_1 = torch.nn.MSELoss()
optimizer_1 = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.02)
scheduler_1 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_1, T_max=32, eta_min=0, last_epoch=-1)

# Convert training data to PyTorch tensors
X_train_1 = np.concatenate((X_2fold, X_3fold), axis=0)
Y_train_1 = np.concatenate((Y_2fold, Y_3fold), axis=0)
X_train_1_tensor = torch.from_numpy(X_train_1).to(device)
Y_train_1_tensor = torch.from_numpy(Y_train_1).to(device)

# Assuming you have RainTrainDataset defined
trainset_1 = RainTrainDataset(X_train_1, Y_train_1)
trainloader_1 = DataLoader(trainset_1, batch_size=batch_size)

In [8]:
# Training loop with early stopping
num_epochs = 60
early_stopping_patience = 100

best_test_loss_1 = 1000000
model_path = 'Prediction\\0219BLSTM_GRU136.pth'
for epoch in range(1, num_epochs + 1):

    print(" %d epoch ... " % epoch)
    model.train()
    for i, (hist, target) in enumerate(trainloader_1, 1):
        hist = hist.float()
        output = model(hist).to(device)
        output = output.reshape(-1, 1)
        target = target.float().reshape(-1, 1).to(device)
        loss = criterion_1(output, target)
        optimizer_1.zero_grad()
        loss.backward()
        optimizer_1.step()
        scheduler_1.step()

    with torch.no_grad():
        _, _, train_MSE, _, _ = evaluate_model(X_train_1_tensor, Y_train_1_tensor, model, criterion_1)

        _, _, test_MSE,test_MAE, test_R2 = evaluate_model(X1fold_tensor, Y1fold_tensor, model, criterion_1)

        if test_MSE < best_test_loss_1:
            best_test_loss_1 = test_MSE
            torch.save(model.state_dict(), model_path)
            print('best test R2 at this time', test_R2)

# Load the best model
best_model = ChloBLSTMGRU(input_size=feat_num, hidden_size=hidden_dim, output_size=pred_len).to(device)
best_model.load_state_dict(torch.load(model_path))

# Test the model
with torch.no_grad():
    test_true_1fold, test_pred_1fold, test_MSE_1fold,test_MAE_1fold, test_R2_1fold = evaluate_model(
        X1fold_tensor, Y1fold_tensor, best_model, criterion_1)

    print(f'test 1 fold MSE: {test_MSE_1fold}')
    print(f'test 1 fold R2: {test_R2_1fold}')
    plt.figure(3)
    plt.scatter(test_pred_1fold, test_true_1fold, marker='+', color='blue', s=40)
    plt.plot(np.array([0, 30]), np.array([0, 30]))
    plt.xlabel('pred')
    plt.ylabel('true')
    plt.savefig('0219BLSTM_GRU136test.jpg')

    train_true_1fold, train_pred_1fold, train_MSE_1fold, train_R2_1fold = evaluate_model(
        X_train_1_tensor, Y_train_1_tensor, best_model, criterion_1)

    print(f'train 1 fold MSE: {train_MSE_1fold}')
    print(f'train 1 fold R2: {train_R2_1fold}')
    plt.figure(5)
    plt.scatter(train_pred_1fold, train_true_1fold, marker='+', color='blue', s=40)
    plt.plot(np.array([0, 20]), np.array([0, 25]))
    plt.xlabel('pred')
    plt.ylabel('true')
    plt.savefig('0219BLSTM_GRU136train.jpg')

 1 epoch ... 
best test R2 at this time 0.2596908208969315
 2 epoch ... 
best test R2 at this time 0.2841515042072743
 3 epoch ... 
 4 epoch ... 
best test R2 at this time 0.29344535480243905
 5 epoch ... 
 6 epoch ... 
best test R2 at this time 0.30642660603480065
 7 epoch ... 
 8 epoch ... 
best test R2 at this time 0.3155457790576678
 9 epoch ... 
 10 epoch ... 
best test R2 at this time 0.3160023469608697
 11 epoch ... 
 12 epoch ... 


KeyboardInterrupt: 